This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [ ]:
!pip install lancedb --quiet
!pip install pandas --quiet

In [1]:
# all the needed imports

import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from lancedb.pydantic import vector, LanceModel
import lancedb
import csv
import openai
import pandas as pd

In [3]:
os.environ["OPENAI_API_KEY"] = "voc-61781640212667736367506729c66ccebd53.09187229"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 4000)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Generating Real Estate Listings

In [6]:
# generate 10 different real estates and save them into Listings.txt

data_gen_template="""
Generate CSV-formatted descriptions for in total {num_properties} different imaginary properties. 
For each property, come up with a name and generate a detailed description. 
Each property will have the following fields in the CSV: property name (string), neighborhood (string), price (float), bedrooms (int), bathrooms (int), house size (float), description (string), neighborhood description (string).
Be creative and diverse in your descriptions, ensuring that each property has unique features and style. 
CSV format is a must. 
Don't use enumerations or unnecessary newlines.

Here is an Example:

Neighborhood: Green Oaks
Price: $800000.0 
Bedrooms: 3
Bathrooms: 2
House Size: 2000.0 sqft
Description: "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."
Neighborhood Description: "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
"""
data_gen_prompt = PromptTemplate.from_template(data_gen_template)

data = llm(data_gen_prompt.format(num_properties=10))

rows = [line.split(",") for line in data.strip().split("\n")]
file_name = "Listings.txt"

with open(file_name, mode="w") as file:
    file.write(data.strip())

## Storing Listings in a Vector Database

In [7]:
# get real estates from .txt

data = []
file_name = "Listings.txt"

with open(file_name, mode="r") as f:
    reader = csv.DictReader(f)
    for r in reader:
        row = {
            "property_name": str(r["Property Name"]),
            "neighborhood": str(r["Neighborhood"]),
            "price": float(r["Price"].replace("$", "")),
            "bedrooms": int(r["Bedrooms"]),
            "bathrooms": float(r["Bathrooms"]),
            "house_size": float(r["House Size"].replace(" sqft", "")),
            "description": str(r["Description"]),
            "neighborhood_description": str(r["Neighborhood Description"])
        }
        data.append(row)
        
print(str(data[0]))

{'property_name': 'Sunset Haven', 'neighborhood': 'Sunset Hills', 'price': 950000.0, 'bedrooms': 4, 'bathrooms': 3.0, 'house_size': 2500.0, 'description': 'Escape to the luxurious Sunset Haven in the prestigious Sunset Hills neighborhood. This 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and a private backyard oasis with a sparkling pool and spa. The spacious master suite offers a tranquil retreat with a spa-like bathroom and walk-in closet. Enjoy breathtaking sunset views from the rooftop deck, perfect for entertaining guests or relaxing in style.', 'neighborhood_description': 'Sunset Hills is known for its upscale living, with tree-lined streets, top-rated schools, and exclusive country clubs. Residents can enjoy shopping at the nearby Sunset Plaza, dining at gourmet restaurants, and easy access to hiking trails in the surrounding hills.'}


In [8]:
# create embeddings for the real estates

for row in data:
    response = openai.Embedding.create(input=str(row), model="text-embedding-ada-002")
    row["vector"] = response['data'][0]['embedding']

In [9]:
len(data[0]["vector"])

1536

In [11]:
# create LanceDB and table "listings" containing the real estates

class Listings(LanceModel):
    vector: vector(1536)
    neighborhood: str
    price: float
    bedrooms: int
    bathrooms: int
    house_size: float
    description: str
    neighborhood_description: str

db = lancedb.connect("~/.lancedb")
table_name = "listings"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=Listings)

table.add(data)

## Building the User Preference Interface

In [12]:
# questions and example answers

questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
    ]

## Searching Based on Preferences

In [17]:
# generate embeddings for user answer

def generate_answer_embedding(answers):
    clients_wishes = " ".join(answers)
    return openai.Embedding.create(input=clients_wishes, model="text-embedding-ada-002")['data'][0]['embedding']

# generate embeddings for example answers and search for the best fitting real estate

def get_matching_real_estate(table, answer_embedding):
    properties = table.search(answer_embedding).limit(3).to_pandas()
    columns = ['neighborhood', 
           'price', 
           'bedrooms', 
           'bathrooms', 
           'house_size', 
           'description', 
           'neighborhood_description']
    return properties[columns].iloc[0].to_dict()

In [18]:
# get best fitting real estate for example answers

answer_embedding = generate_answer_embedding(answers)
best_fitting_real_estate = get_matching_real_estate(table, answer_embedding)#fitting_real_estates[columns].iloc[0].to_dict()

best_fitting_real_estate

{'neighborhood': 'Old Town',
 'price': 600000.0,
 'bedrooms': 2,
 'bathrooms': 1,
 'house_size': 1600.0,
 'description': 'Step back in time in the Historic Haven located in the picturesque Old Town neighborhood. This 2-bedroom, 1-bathroom home features historic charm, original hardwood floors, and a cozy fireplace with a vintage mantel. The quaint kitchen offers retro appliances and a breakfast nook, while the backyard garden provides a peaceful retreat with a pergola and flower beds. Embrace the nostalgia of yesteryear in this historic haven, where timeless elegance and character abound in every corner.',
 'neighborhood_description': 'Old Town is a historic neighborhood known for its cobblestone streets, Victorian architecture, and charming boutiques. Residents can enjoy walking to nearby museums, art galleries, and antique shops, as well as easy access to historic landmarks and cultural attractions. With a strong sense of community and a focus on preserving the past, Old Town offers 

## Personalizing Listing Descriptions

In [20]:
# get the pros and cons of the submitted best fitting real estate regarding to the clients answers

def get_pros_and_cons(real_estate, clients_wishes):

    system_message = f"""You receive a property and the customer's wishes.
    The price is in $ and the house size in sqft.

    =====

    PROPERTY: {real_estate}

    =====

    CLIENTS WISHES: {clients_wishes}

    =====
    
    List all the property features that meet the clients wishes and justify that.
    List all the property features that do not meet the clients wishes and justify that.

    """
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message}
            ],
        max_tokens=500
    )["choices"][0]["message"]["content"]

In [21]:
# get the pros and cons of the submitted best fitting real estate regarding to the clients example answers

pros_and_cons = get_pros_and_cons(best_fitting_real_estate, answers)
print(pros_and_cons)

Property features that meet the client's wishes:
1. The property has 3 bedrooms as requested by the client.
2. The kitchen is described as quaint and offers retro appliances, which may be considered cozy and spacious by the client.
3. There is a cozy living room with a fireplace, fulfilling the client's desire for a cozy living space.
4. The neighborhood, Old Town, is described as historic and peaceful, aligning with the client's preference for a quiet neighborhood.
5. Old Town is known for its charm and proximity to boutiques, museums, and art galleries, meeting the client's desire for convenient shopping options and access to cultural attractions.
6. The property features a backyard garden with flower beds, providing space for gardening.
7. While the property does not mention a two-car garage specifically, it is common in this neighborhood to have parking options, and the client may be able to find a property with a two-car garage in this area.
8. The property has historic charm and 

In [23]:
# get final answer of the real estate agent

def get_final_answer(pros_and_cons):
    system_message = f"""You are a real estate agent who sells houses to clients.
        Here are some information about a Property and its features:

        {pros_and_cons}

        Sell the property to the client, but only mention the Features that were explicitely mentioned and meet the clients wishes.
        Answer in continous text.
        """

    return openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_message}
                ],
                max_tokens=500
            )["choices"][0]["message"]["content"]

In [24]:
# get final answer of the real estate agent regarding the example answers

final_answer = get_final_answer(pros_and_cons)
print(final_answer)

I have a wonderful property that I believe meets most of your desired features. This charming house is located in the historic neighborhood of Old Town, known for its peaceful and historic charm. The property has 3 bedrooms, a cozy living room with a fireplace, and a quaint kitchen with retro appliances that you may find cozy and spacious. The neighborhood offers convenient shopping options with boutiques, museums, and art galleries nearby. Additionally, there is a lovely backyard garden with flower beds for your gardening enjoyment. While the property may not have a two-car garage specifically mentioned, it is common in this neighborhood to have parking options available. The house has historic charm and character that you may appreciate. If you are interested in exploring this property further, I can provide more details and schedule a viewing for you.


## Deliverables and Testing

### Questions 
- How big do you want your house to be?
- What are 3 most important things for you in choosing this property?
- Which amenities would you like?
- Which transportation options are important to you?
- How urban do you want your neighborhood to be?  

In [25]:
# real estate pipeline to search for the best fit

def get_real_estate_recommandation(real_estate_db, clients_wishes):
    
    clients_wishes_embedding = generate_answer_embedding(clients_wishes)
    real_estate = get_matching_real_estate(real_estate_db, clients_wishes_embedding)
    pros_and_cons = get_pros_and_cons(real_estate, clients_wishes)
    return get_final_answer(pros_and_cons)

In [26]:
answers_1 = [
    "A house with at least 2,300 square feet, offering plenty of space for a comfortable and luxurious lifestyle.",
    "Top ratet schools, walk in closet, and convenient shopping options.",
    "I need a gourmet kitchen, a private pool and spa.",
    "I have a car so i need good streets.",
    "I love it urban with shopping possibilities, gourmet restaurants."
]

print(get_real_estate_recommandation(table, answers_1))

I'm thrilled to present to you a fantastic property that perfectly aligns with your wishes and requirements. This stunning house in the Sunset Hills neighborhood boasts a spacious 2,500 sqft, exceeding your minimum size preference of 2,300 sqft. It features top-rated schools in the area, ensuring excellent educational opportunities for your family. The master suite includes a luxurious walk-in closet, perfect for organizing your wardrobe. Additionally, the gourmet kitchen offers high-end appliances and finishes, catering to your desire for a top-notch culinary space. Outside, you'll find a private backyard oasis with a refreshing pool and spa, ideal for relaxation and entertainment. The Sunset Hills neighborhood also provides convenient access to shopping at Sunset Plaza and gourmet restaurants, creating a vibrant urban environment. I believe this property embodies everything you're looking for in a dream home. Let's schedule a viewing soon to experience its charm firsthand!


In [27]:
answers_2 = [
    "A compact but functional apartment with two bedrooms and a home office.",
    "Great public transport options, an active community, and walking distance to restaurants.",
    "A balcony with a view, energy-efficient appliances, and pet-friendly spaces.",
    "Close to subway stations, walking distance to shops, and bike lanes throughout the area.",
    "A more rural neighborhood with a quiet atmosphere but within commuting distance to the city."
]

print(get_real_estate_recommandation(table, answers_2))

I am excited to present to you a wonderful property that perfectly aligns with your preferences and requirements. This fantastic 2-bedroom loft is located in the Downtown Loft neighborhood, offering a compact yet functional living space that includes a home office, meeting your need for a practical apartment with two bedrooms and workspace options. 

Furthermore, this property boasts great public transport options, allowing for easy commuting around the city, and is situated in an active community with trendy restaurants and shops within walking distance, providing you with the vibrant atmosphere and proximity to dining establishments that you desire. 

In addition, you will love the rooftop terrace with stunning city views, offering a balcony with a view where you can relax and take in the scenery. This property also has the potential to include energy-efficient appliances and can be modified to accommodate your furry friends, making it a pet-friendly space for you and your beloved co

In [28]:
answers_3 = [
    "A large family home with five bedrooms, a playroom for kids, and an open-plan living space.",
    "Safe neighborhood with a low crime rate, proximity to medical facilities, and recreational parks.",
    "A fireplace, a finished basement, and a kitchen with a large island and high-end appliances.",
    "Easy access to highways, availability of carpool lanes, and options for electric vehicle charging.",
    "A lively neighborhood with diverse dining options, parks, and cultural attractions."
]

print(get_real_estate_recommandation(table, answers_3))

I have a fantastic property in the Old Town neighborhood that meets many of your wishes. The area is known for its safety and low crime rate, with medical facilities nearby for added convenience. The house features a cozy fireplace for those chilly evenings. Additionally, Old Town offers a vibrant neighborhood with charming boutiques, museums, and art galleries to explore. If you're interested in a property with these features, this could be the perfect place for you!
